In [51]:
import cv2
import math
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
import csv
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import plot_confusion_matrix, confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle


from time import time
from mpl_toolkits.mplot3d import Axes3D

import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import Model
from keras.callbacks import EarlyStopping


#####################################################################################
# Initiate mediapipe 


mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic

In [29]:
cap = cv2.VideoCapture(1)  #Might need to change from 1 to a different number... default system camera is usually 0.
## Setup mediapipe instance
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
        image.flags.writeable = False                  # Image is no longer writeable
        results = holistic.process(image)                 # Make prediction
        image.flags.writeable = True                   # Image is now writeable 
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
        image.flags.writeable = False
      
        
        # Extract landmarks
        try:
            landmarks_pose = results.pose_landmarks.landmark
            landmarks_left_hand = results.left_hand_landmarks.landmark
            landmarks_right_hand = results.right_hand_landmarks.landmark

        except:
            pass
        
        
         # Render detections
        #mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) 
        # Draw pose connections
        #mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) 
        # Draw left hand connections
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) 
        # Draw right hand connections  
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)        
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [14]:
len(landmarks_pose)

33

In [15]:
len(landmarks_left_hand)

21

In [16]:
len(landmarks_right_hand)

21

In [88]:
classes = ['class']
#for val in range(1, 33+1):
#    classes += ['pose_x{}'.format(val), 'pose_y{}'.format(val), 'pose_z{}'.format(val), 'pose_visibility{}'.format(val)]
for val in range(1, 21+1):
    classes += ['lh_x{}'.format(val), 'lh_y{}'.format(val), 'lh_z{}'.format(val)]
for val in range(1, 21+1):
    classes += ['rh_x{}'.format(val), 'rh_y{}'.format(val), 'rh_z{}'.format(val)]

In [89]:
with open('coords_4.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(classes)

In [134]:
class_name = "assist" # Enter name of classification
CSV_FILE = 'coords_4.csv'
sequence_length = 50 # length of recording

In [135]:
seq_len = sequence_length 

cap = cv2.VideoCapture(1)  #Might need to change from 1 to a different number... default system camera is usually 0.
## Setup mediapipe instance
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            ret, frame = cap.read()

            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
            image.flags.writeable = False                  # Image is no longer writeable
            results = holistic.process(image)                 # Make prediction
            image.flags.writeable = True                   # Image is now writeable 
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
            image.flags.writeable = False


            # Extract landmarks
            try:
                #pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
                lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
                rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
                #landmarks = np.concatenate([pose, lh, rh])
                landmarks = np.concatenate([ lh, rh])
                landmarks = np.append(class_name,landmarks)


                #Export to CSV
                with open(CSV_FILE, mode='a', newline='') as f:
                    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                    csv_writer.writerow(landmarks)

            except:
                pass



            cv2.putText(image, 'Collecting frames for {}, Timer {}'.format(class_name, seq_len), (15,12), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
             # Render detections
            #mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) 
            # Draw pose connections
            #mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) 
            # Draw left hand connections
            mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) 
            # Draw right hand connections  
            mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)        

            cv2.imshow('Mediapipe Feed', image)
            seq_len = seq_len - 1
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
            if seq_len < 1:
                break
        cap.release()
        cv2.destroyAllWindows()